In [2]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask
import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'postgresql+psycopg2://localhost/test_db'
app.config['SQLALCHEMY_COMMIT_ON_TEARDOWN'] = True
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class Customer(db.Model):
    __tablename__ = 'customers'
    id = db.Column(db.Integer, primary_key=True)
    first_name = db.Column(db.String(100), nullable=False)
    last_name = db.Column(db.String(100), nullable=False)
    username = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(200), nullable=False)
    address = db.Column(db.String(200), nullable=False)
    town = db.Column(db.String(50), nullable=False)
    created_on = db.Column(db.DateTime(), default=datetime.datetime.now())
    updated_on = db.Column(db.DateTime(), default=datetime.datetime.now(), onupdate=datetime.datetime.now())

    def __init__(self, **kwargs):
        super(Customer, self).__init__(**kwargs)

    def __repr__(self):
        return '<Customer %r>' % self.id

class Item(db.Model):
    __tablename__ = 'items'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    cost_price = db.Column(db.Numeric(10, 2), nullable=False)
    selling_price = db.Column(db.Numeric(10, 2),  nullable=False)
    quantity = db.Column(db.Integer(), nullable=False)
    db.CheckConstraint('quantity > 0', name='quantity_check')

    def __init__(self, **kwargs):
        super(Item, self).__init__(**kwargs)

    def __repr__(self):
        return '<Item %r>' % self.name

class Order(db.Model):
    __tablename__ = 'orders'
    id = db.Column(db.Integer, primary_key=True)
    customer_id = db.Column(db.Integer, db.ForeignKey('customers.id'))
    date_placed = db.Column(db.DateTime(), default=datetime.datetime.now())
    date_shipped = db.Column(db.DateTime())
    customer = db.relationship('Customer', backref='orders')

    def __init__(self, **kwargs):
        super(Order, self).__init__(**kwargs)

    def __repr__(self):
        return '<Order %r>' % self.id

class OrderLine(db.Model):
    __tablename__ = 'orderlines'
    id = db.Column(db.Integer, primary_key=True)
    order_id = db.Column(db.Integer, db.ForeignKey('orders.id'))
    item_id = db.Column(db.Integer, db.ForeignKey('items.id'))
    quantity = db.Column(db.Integer)
    order = db.relationship('Order', backref='orderlines')
    item = db.relationship('Item', backref='orderlines')

    def __init__(self, **kwargs):
        super(OrderLine, self).__init__(**kwargs)

    def __repr__(self):
        return '<OrderLine %r>' % self.quantity

db.create_all()

In [3]:
c1 = Customer(first_name='Toby',
                      last_name='Miller',
                      username='tmiller',
                      email='tmiller@example.com',
                      address='1662 Kinney Street',
                      town='Wolfden'
                      )

c2 = Customer(first_name='Scott',
              last_name='Harvey',
              username='scottharvey',
              email='scottharvey@example.com',
              address='424 Patterson Street',
              town='Beckinsdale'
              )
c3 = Customer(
    first_name="John",
    last_name="Lara",
    username="johnlara",
    email="johnlara@mail.com",
    address="3073 Derek Drive",
    town="Norfolk"
)

c4 = Customer(
    first_name = "Sarah",
    last_name = "Tomlin",
    username = "sarahtomlin",
    email = "sarahtomlin@mail.com",
    address = "3572 Poplar Avenue",
    town = "Norfolk"
)

c5 = Customer(first_name = 'Toby',
              last_name = 'Miller',
              username = 'tmiller',
              email = 'tmiller@example.com',
              address = '1662 Kinney Street',
              town = 'Wolfden'
              )

c6 = Customer(first_name = 'Scott',
              last_name = 'Harvey',
              username = 'scottharvey',
              email = 'scottharvey@example.com',
              address = '424 Patterson Street',
              town = 'Beckinsdale'
              )

db.session.add_all([c1, c2, c3, c4, c5, c6])
db.session.commit()

In [6]:
i1 = Item(name = 'Chair', cost_price = 9.21, selling_price = 10.81, quantity = 5)
i2 = Item(name = 'Pen', cost_price = 3.45, selling_price = 4.51, quantity = 3)
i3 = Item(name = 'Headphone', cost_price = 15.52, selling_price = 16.81, quantity = 50)
i4 = Item(name = 'Travel Bag', cost_price = 20.1, selling_price = 24.21, quantity = 50)
i5 = Item(name = 'Keyboard', cost_price = 20.1, selling_price = 22.11, quantity = 50)
i6 = Item(name = 'Monitor', cost_price = 200.14, selling_price = 212.89, quantity = 50)
i7 = Item(name = 'Watch', cost_price = 100.58, selling_price = 104.41, quantity = 50)
i8 = Item(name = 'Water Bottle', cost_price = 20.89, selling_price = 25, quantity = 50)

db.session.add_all([i1, i2, i3, i4, i5, i6, i7, i8])
db.session.commit()

o1 = Order(customer = c1)
o2 = Order(customer = c1)

line_item1 = OrderLine(order = o1, item = i1, quantity =  3)
line_item2 = OrderLine(order = o1, item = i2, quantity =  2)
line_item3 = OrderLine(order = o2, item = i1, quantity =  1)
line_item4 = OrderLine(order = o2, item = i2, quantity =  4)

db.session.add_all([o1, o2])
# db.session.add_all([line_item1, line_item2,line_item3,line_item4])

db.session.new
db.session.commit()


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "items_name_key"
DETAIL:  Key (name)=(Chair) already exists.

[SQL: INSERT INTO items (name, cost_price, selling_price, quantity) VALUES (%(name)s, %(cost_price)s, %(selling_price)s, %(quantity)s) RETURNING items.id]
[parameters: {'name': 'Chair', 'cost_price': 9.21, 'selling_price': 10.81, 'quantity': 5}]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [24]:
db.session.remove()
i1 = Item(name = 'Tool', cost_price = 9.21, selling_price = 10.81, quantity = 5)
i2 = Item(name = 'tolls', cost_price = 3.45, selling_price = 4.51, quantity = 3)
#
i1 = db.session.query(Item).filter(Item.name == 'Tool').first()
i2 = db.session.query(Item).filter(Item.name == 'tolls').first()

o3 = Order(customer = c2)
orderline1 = OrderLine(item = i1, quantity = 5)
orderline2 = OrderLine(item = i2, quantity = 10)
#
o3.orderlines.append(orderline1)
o3.orderlines.append(orderline2)

db.session.add_all([o3])
db.session.commit()